In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_recall_fscore_support,classification_report, confusion_matrix
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
from sklearn.linear_model import Ridge
from sklearn.preprocessing import Normalizer

In [19]:
#These are the functions I will use for result visualisation
def plot_confusion_matrix(y_test,y_predicted,labels):
    cm = confusion_matrix(y_test, y_predicted,labels =labels)

    figsize = (10,7)
    df_cm = pd.DataFrame(
        cm, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    return

def evaluate_classifier(pipeline, x_test, y_test):

    y_predicted = pipeline.predict(x_test)

    report  = classification_report(y_test, np.round(y_predicted))
    print(report)
    #plot_confusion_matrix(y_test, y_predicted, class_names)
    return

In [2]:
train_data = pd.read_csv('data/train.csv', index_col='customer')
test_data = pd.read_csv('data/test.csv', index_col='customer')

In [3]:
train_dummies = pd.get_dummies(train_data, columns=['category', 'is_pep', 'nationality'])
test_dummies = pd.get_dummies(test_data, columns=['category', 'is_pep', 'nationality'])

In [4]:
x_train = train_dummies.drop(['suspicious'], axis=1)
y_train = pd.DataFrame(train_dummies['suspicious'])
x_test = test_dummies.copy()
x_test['nationality_117'] = 0

In [5]:
pipeline = Pipeline([
    ('normalizer', Normalizer()), 
    ('ridge', Ridge(alpha=1.0))
])

In [6]:
pipeline.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('normalizer', Normalizer(copy=True, norm='l2')), ('ridge', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])

In [7]:
y_pred = pipeline.predict(x_test)

In [20]:
evaluate_classifier(pipeline, x_train, y_train)

             precision    recall  f1-score   support

          0       0.99      1.00      1.00    990913
          1       0.00      0.00      0.00      9087

avg / total       0.98      0.99      0.99   1000000



C:\Users\Marku\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
